# Apache Spark Guide

This is an Apache Spark Guide that will be featured on the [Algotrading101 blog](https://algotrading101.com/learn/).
If you want the explanation of what the code does I'd suggest you to visit the blog article that will be linked [here](https://algotrading101.com/learn/pyspark-guide/) upon release.

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Create the Session
spark = SparkSession.builder\
    .master("local")\
    .appName("PySpark Tutorial")\
    .getOrCreate()

In [ ]:
sc = spark.sparkContext
rdd = sc.textFile('../input/stockmarketdatafrom1996to2020/Data/Data/FB/FB.csv')
rdd.take(2)

In [ ]:
from pathlib import Path
contents = list(Path('../input/stockmarketdatafrom1996to2020/Data/Data').iterdir())
#print(contents)

In [ ]:
stock_1 = spark.read.csv('../input/stockmarketdatafrom1996to2020/Data/Data/AAPL/AAPL.csv', inferSchema=True, header=True)
stock_1.show(5)

In [ ]:
stock_2 = spark.read.csv('../input/stockmarketdatafrom1996to2020/Data/Data/MSFT/MSFT.csv', inferSchema=True, header=True)
stock_2.show(5)

In [ ]:
stock_1.printSchema()

In [ ]:
stock_1.select("Close").show(10)

In [ ]:
from pyspark.sql import functions as F
stock_1.filter(F.col("Close")>148.00).select("Date","Close").show(10)

In [ ]:
rdd = rdd.map(lambda line: line.split(","))
rdd.top(5)

In [ ]:
num = sc.parallelize([23, 1, 4, 5, 6, 7])
num_sum = num.reduce(lambda a,b:a+b)
print(num_sum)

In [ ]:
from pyspark.sql import Row

header = rdd.first()
stock_3 = rdd.filter(lambda line: line != header)\
             .map(lambda line: Row(date=line[0],
                                   open=line[1],
                                   high=line[2],
                                   low=line[3],
                                   close=line[4],
                                   adj_close=line[5],
                                   volume=line[6])).toDF()
stock_3.show(5)

In [ ]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

input_1 = stock_1.select("Adj Close")
input_1.show(5)
input_2 = stock_2.select("Adj Close")
#######################

input_1 = input_1.withColumnRenamed("Adj Close","label")
input_2 = input_2.withColumnRenamed("Adj Close","feature")

input_data = input_1.join(input_2)
input_data.show(5)

In [ ]:
assembler = VectorAssembler(
    inputCols=["feature"],
    outputCol="features")

input_data = assembler.transform(input_data)

standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaler = standardScaler.fit(input_data.select("features"))

df = scaler.transform(input_data)
df.show(5)

In [ ]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = df.randomSplit([.8,.2], seed=42)

reg = LinearRegression(labelCol="label", featuresCol="features_scaled", maxIter=5)
model = reg.fit(train_data)

In [ ]:
# Predict test_data
predicted = model.transform(test_data)

# Take predictions and the true label - zip them
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])
pred_lab = predictions.zip(labels).collect()

# Print out first 5  predictions
pred_lab[:5]

In [ ]:
# Model coefficients
print(model.coefficients)

# Intercept
print(model.intercept)

# RMSE
print(model.summary.rootMeanSquaredError)